In [11]:
## Packages
using LinearAlgebra
using Colors
using Printf

## Basic functions
function DeleteDuplicates(v::Array{T,1})::Array{T,1} where T <: Any
    w=Array{eltype(v),1}()
    if (length(v)>0)
        for e ∈ v
            if e ∉ w
                push!(w,e)
            end
        end
    end
    return w
end

function OrthogonalVector(v::Array{T,1}) where T <: Real
    if(norm(v)==0)
        error("zero vector")
    else
        v₁=normalize(v)
        i=argmin(abs.(v))
        v₂=zeros(length(v))
        v₂[i]=1.0
        v₂=normalize(v₂-dot(v₁,v₂)*v₁)
        return v₂
    end
end

function Circumcenter(point₁,point₂,point₃)
    l₁²=(norm(point₂-point₃))^2
    l₂²=(norm(point₃-point₁))^2
    l₃²=(norm(point₁-point₂))^2
    c₁=l₁²*(l₂²+l₃²-l₁²)
    c₂=l₂²*(l₃²+l₁²-l₂²)
    c₃=l₃²*(l₁²+l₂²-l₃²)
    center=(c₁*point₁+c₂*point₂+c₃*point₃)/(c₁+c₂+c₃)
    return center
end

## Types
abstract type JuliRay end
abstract type Object <: JuliRay end

## Primitive shapes
struct Empty <: Object end
struct Sphere <: Object
    center :: Array{Float64,1}
    radius :: Float64
    Sphere(center,radius) = (radius ≠ 0.0) ? new(center,abs(radius)) : Empty
end
struct Cylinder <: Object
    end1 :: Array{Float64,1}
    end2 :: Array{Float64,1}
    radius :: Float64
    Cylinder(end1,end2,radius) = (norm(end2-end1)*radius ≠ 0) ? new(end1,end2,abs(radius)) : Empty
end
struct Cone <: Object
    end1 :: Array{Float64,1}
    end2 :: Array{Float64,1}
    radius :: Float64
    Cone(end1,end2,radius) = (norm(end2-end1)*radius ≠ 0) ? new(end1,end2,abs(radius)) : Empty
end
struct Box <: Object
    vertex1 :: Array{Float64,1}
    vertex2 :: Array{Float64,1}
    Box(vertex1,vertex2) = (norm(vertex2-vertex1) ≠ 0) ? new(vertex1,vertex2) : Empty
end
struct Disc <: Object
    center :: Array{Float64,1}
    normal :: Array{Float64,1}
    radius :: Float64
    Disc(center,normal,radius) =
    if(norm(normal) == 0)
        error()
    elseif(radius == 0)
        Empty
    else
        new(center,normal,abs(radius))
    end
end
struct Torus <: Object
    radius1 :: Float64
    radius2 :: Float64
    Torus(radius1,radius2) =
    if(radius2 == 0)
        Empty
    elseif(radius1 == 0)
        Sphere([0,0,0],radius2)
    else
        new(radius1,radius2)
    end
end
struct Polygon <: Object
    vertices :: Array{Array{Float64,1},1}
    Polygon(vertices) =
    if(rank(hcat(vertices...)-repeat(+(vertices...)/length(vertices),1,length(vertices))) ≠ 2)
        Empty
    else
        new(vertices)
    end
end

struct csgUnion <: Object
    objects :: Array{Object,1}
    csgUnion(objects) =
    if(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 0)
        Empty
    elseif(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 1)
        objects[1]
    else
        new(DeleteDuplicates(deleteat!(objects, objects.== Empty)))
    end
end
struct csgIntersection <: Object
    objects :: Array{Object,1}
    csgIntersection(objects) =
    if(Empty ∈ objects)
        Empty
    elseif(length(DeleteDuplicates(objects)) == 0)
        Empty
    elseif(length(DeleteDuplicates(objects)) == 1)
        objects[1]
    else
        new(DeleteDuplicates(objects))
    end
end
struct csgMerge <: Object
    objects :: Array{Object,1}
    csgMerge(objects) =
    if(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 0)
        Empty
    elseif(length(DeleteDuplicates(deleteat!(objects, objects.== Empty))) == 1)
        objects[1]
    else
        new(DeleteDuplicates(deleteat!(objects, objects.== Empty)))
    end
end
struct csgDifference <: Object
    objects :: Array{Object,1}
    csgDifference(objects) =
    if(length(objects) ≠ 2)
        error("だめです")
    elseif(objects[1] == Empty)
        Empty
    elseif(objects[2] == Empty)
        objects[1]
    else
        new(objects)
    end
end

# Transform object to object
struct rgbColor <:Object
    object :: Object
    color :: Array{Float64,1}
    rgbColor(object,color) = (object ≠ Empty) ? new(object,color) : Empty
end
struct AffineTransform <:Object
    object :: Object
    A :: Array{Float64,2}
    b :: Array{Float64,1}
    AffineTransform(object,A,b) = (object ≠ Empty && det(A) ≠ 0) ? new(object,A,b) : Empty
end




## 文字列変換
function vector2pov(x::Array{T,1}) where T <: Real
    "<"*repr(x)[2:end-1]*">"
end

function object2pov(sphere :: Sphere)
    return "sphere{"*vector2pov(sphere.center)*","*repr(sphere.radius)*"}"
end
function object2pov(cylinder :: Cylinder)
    return "cylinder{"*vector2pov(cylinder.end1)*","*vector2pov(cylinder.end2)*","*repr(cylinder.radius)*"}"
end
function object2pov(cone :: Cone)
    return "cone{"*vector2pov(cone.end1)*","*repr(cone.radius)*","*vector2pov(cone.end2)*",0}"
end
function object2pov(box :: Box)
    return "box{"*vector2pov(box.vertex1)*","*vector2pov(box.vertex2)*"}"
end
function object2pov(disc :: Disc)
    return "disc{"*vector2pov(disc.center)*","*vector2pov(disc.normal)*","*repr(disc.radius)*"}"
end
function object2pov(torus :: Torus)
    return "torus{"*repr(torus.radius1)*","*repr(torus.radius2)*"}"
end
function object2pov(polygon :: Polygon)
    n=length(polygon.vertices)
    return "polygon{"*repr(n)* (*(reshape([(repeat([","],n),vector2pov.(polygon.vertices))[i][j] for i in 1:2, j in 1:n],2n)...))*"}"
end
# csg
function object2pov(csg :: csgUnion)
    return "union{"* *(object2pov.(csg.objects)...)*"}"
end
function object2pov(csg :: csgIntersection)
    return "intersection{"* *(object2pov.(csg.objects)...)*"}"
end
function object2pov(csg :: csgMerge)
    return "merge{"* *(object2pov.(csg.objects)...)*"}"
end
function object2pov(csg :: csgDifference)
    return "difference{"* *(object2pov.(csg.objects)...)*"}"
end
# Transform
function object2pov(rgbcolor :: rgbColor)
    return "object{"*object2pov(rgbcolor.object)*" pigment{rgb"*vector2pov(rgbcolor.color)*"}}"
end
function object2pov(affinetransform :: AffineTransform)
    return "object{"*object2pov(affinetransform.object)*" matrix"*vector2pov(vcat(reshape(affinetransform.A,9),affinetransform.b))*"}"
end

## Verargs objects
function Polygon(vertices...)
    return Polygon([vertices...])
end
function csgUnion(objects::Object...)
    return csgUnion([objects...])
end
function csgIntersection(objects::Object...)
    return csgIntersection([objects...])
end
function csgMerge(objects::Object...)
    return csgMerge([objects...])
end
function csgDifference(object1,object2)
    return csgDifference([object1,object2])
end


## Compound objects
function Arrow(end1,end2,radius)
    n=normalize(end2-end1)
    end3=end2-6*radius*n
    return csgUnion(Cylinder(end1,end3,radius),Cone(end3,end2,2*radius))
end
function Torus(center :: Array{Float64,1},normal :: Array{Float64,1},radius1::Float64,radius2::Float64)
    e₁=normalize(normal)
    e₂=OrthogonalVector(e₁)
    e₃=cross(e₁,e₂)
    A=hcat(e₃,e₁,e₂)
    return AffineTransform(Torus(radius1,radius2),A,center)
end
function Torus(point1 :: Array{Float64,1},point2 :: Array{Float64,1},point3 :: Array{Float64,1},radius::Float64)
    center=Circumcenter(point1,point2,point3)
    normal=cross(point1-point2,point2-point3)
    R=norm(point1-center)
    return Torus(center,normal,R,radius)
end

# Rendering
function render(obj::T;name="new", index::Int=0, width::Int=500, height::Int=500) where T <: Object
    if(0<index<1000000)
        Index="_"*(@sprintf "%06d" index)
    elseif(index==0)
        Index=""
    else
        error("index must be non-negative and less than 1000000")
    end
    if(endswith(name,".pov"))
        Name=name[1:end-4]*Index*".pov"
    else
        Name=name*Index*".pov"
    end
    
    str="#version 3.7;\nglobal_settings{assumed_gamma 1.0}\n\n#include \"Hy_constants.inc\"\n#include \"Hy_functions.inc\"\n#include \"Hy_colors.inc\"\n\n#declare Lng=30;\n#declare Lat=30;\n#declare Pers=0.1;\n#declare Zoom=0.9;\n#declare LookAt=<0,0,0>;\n#include \"Hy_camera.inc\"\n\n"
    str=str*object2pov(obj)
    open(Name, "w") do io
      println(io, str)
    end
    run(`povray $Name`)
end

render (generic function with 1 method)

In [12]:
n=10
for iii ∈ 0:(n-1)
    t=iii/n
    θ=π*(1+cos(2π*t))/4
    A=[1 0 0;0 cos(θ) -sin(θ);0 sin(θ) cos(θ)]
    A2=[cos(θ) -sin(θ) 0;sin(θ) cos(θ) 0;0 0 1]
    o=Sphere([0,0,0],0.05)
    x=rgbColor(Arrow([0,0,0],[1,0,0],0.02+0.01*cos(2π*t)),[1,0,0])
    y=rgbColor(Arrow([0,0,0],[0,1,0],0.02+0.01*cos(2π*(t+1/3))),[0,1,0])
    z=rgbColor(Arrow([0,0,0],[0,0,1],0.02+0.01*cos(2π*(t+2/3))),[0,0,1])
    a=Cylinder([0,0,0],[0,0,1],-0.1)
    b=rgbColor(Cone([-1,0,0],[-0.8,0.5,1],-0.1),[1,1,1/2-sin(2π*t)/2])
    c=Box([1,1,1]*0.9,[0,0.7+0.15*sin(2π*t),0.4])
    d=AffineTransform(rgbColor(a,[1,0.2,0.3]),[1 0 0;0 1 0;0 0 1],[0,-0.7,-0.3+0.3cos(2π*t)])
    e=rgbColor(c,[0.2,0.3,0.5])
    f=rgbColor(Torus(0.6,0.15),[0.9,0.1,0.15])
    g=AffineTransform(f,A,[0,0,0])
    h=AffineTransform(rgbColor(Torus(0.6,0.1),[0.2,0.8,0.1]),A2,[0,0,0])
    i=[rgbColor(Disc([1,ii+0.1t,-ii-0.1t],[0.2,-0.4,1],0.1),[0,.8,.8]) for ii ∈ collect(range(-1,1,step=0.1))]
    j=[rgbColor(Torus([1,ii+0.1t,-ii-0.1t],[0.2,-0.4,1],0.1,0.02),[0,.8,.8]) for ii ∈ collect(range(-1,1,step=0.1))]
    p=rgbColor(Polygon([0,0,0.5],[1,0,0],[1,1,-0.5],[0,1,0]),[0,1,1])
    q=Torus([1/2,0,0],[0,1/2,0],[0,0,1.0],0.03)
    obj=csgUnion(o,x,y,z,b,d,e,g,h,i...,j...,q)
    render(obj,name="test_axis",index=iii)
end

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Rendered 250000 of 250000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 500 x 500
----------------------------------------------------------------------------
Pixels:           266000   Samples:          117927   Smpls/Pxl: 0.44
Rays:             383927   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage


----------------------------------------------------------------------------
Box                              40352           22351     55.39
Cone/Cylinder                   114400           31800     27.80
Disc                            112014           12549     11.20
Sphere                            3551            2065     58.15
Torus                           831175          256694     30.88
Torus Bound                     831175          338308     40.70
Bounding Box                   9531772         3449715     36.19
----------------------------------------------------------------------------
Roots tested:                338308   eliminated:               121285
Shadow Ray Tests:            182576   Succeeded:                     0
----------------------------------------------------------------------------
----------------------------------------------------------------------------
Render Time:
  Photon Time:      No photons
  Radiosity Time:   No radiosity
  Trace Time:     

Parser Options
  Input file: test_axis_000003.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....500 by 500 (rows 1 to 500, columns 1 to 500).
  Output file..........test_axis_000003.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:           55
Infinite Objects:          0
Light Sources:        

POV-Ray finished

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

Rendered 250000 of 250000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 500 x 500
----------------------------------------------------------------------------
Pixels:           266000   Samples:          113022   Smpls/Pxl: 0.42
Rays:             379022   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------


Box                              56064           32718     58.36
Cone/Cylinder                   100419           27815     27.70
Disc                            113765           12328     10.84
Sphere                            3556            2107     59.25
Torus                           825372          256529     31.08
Torus Bound                     825372          337670     40.91
Bounding Box                   9088988         3386972     37.26
----------------------------------------------------------------------------
Roots tested:                337670   eliminated:               108616
Shadow Ray Tests:            176792   Succeeded:                     0
----------------------------------------------------------------------------
----------------------------------------------------------------------------
Render Time:
  Photon Time:      No photons
  Radiosity Time:   No radiosity
  Trace Time:       0 hours  0 minutes  0 seconds (0.056 seconds)
              using 16 thread

Parser Options
  Input file: test_axis_000008.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....500 by 500 (rows 1 to 500, columns 1 to 500).
  Output file..........test_axis_000008.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:           55
Infinite Objects:          0
Light Sources:        

POV-Ray finished



In [13]:
o=Sphere([0,0,0],0.05)
render(o,index=123456)

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray new_123456.pov`, ProcessExited(0))